In [ ]:
#!/usr/bin/env python3
"""
Reads separate XYZ files for graphene and water, determines water topology,
centers the water droplet above the graphene sheet (using max graphene Z),
and writes a COMPLETE LAMMPS data file for FLEXIBLE SPC/E water.
Writes non-zero coefficients for Bond/Angle Coeffs. Minimal print output.
"""
import sys
import os
import numpy as np

# --- Configuration ---
# Input Files
graphene_xyz_file   = "/mnt/c/Users/vbarv/Desktop/course/project/lammps/data/graphene_pillars.xyz"
water_xyz_file      = "/mnt/c/Users/vbarv/Desktop/course/project/lammps/data/water_droplet_packed.xyz"

# --- Output File ---
lammps_data_file    = "/mnt/c/Users/vbarv/Desktop/course/project/lammps/data/system_pillars.data"

# --- Centering Configuration ---
desired_z_gap = 4.0 # Keep increased gap

# --- Simulation Box ---
use_hardcoded_box = False
padding = 15.0
z_top_padding = 50.0 # Extra padding for zhi
xlo_hardcoded, xhi_hardcoded = -85.0, 85.0 # Not used if use_hardcoded_box is False
ylo_hardcoded, yhi_hardcoded = -85.0, 85.0 # Not used
zlo_hardcoded, zhi_hardcoded = -5.0, 150.0 # Not used

# --- Atom Types and Masses ---
# O=1, H=2, C=3
atom_types_config = [("O", 15.9994), ("H", 1.008), ("C", 12.011)]
n_atom_types = len(atom_types_config)
symbol_to_type_id = {symbol: i + 1 for i, (symbol, mass) in enumerate(atom_types_config)}
type_id_to_mass = {i + 1: mass for i, (symbol, mass) in enumerate(atom_types_config)}
symbol_to_mass = {symbol: mass for symbol, mass in atom_types_config}

# --- Atom Charges ---
atom_charge_map = { "O": -0.8476, "H": +0.4238, "C": 0.0 }

# --- Topology Types ---
water_bond_type_id = 1; water_angle_type_id = 1
n_bond_types = 1; n_angle_types = 1
n_total_dihedrals = 0; n_total_impropers = 0

# --- Force Field Coefficients (Flexible Water) ---
pair_coeffs = { 1: [0.155,  3.166], 2: [0.0000, 0.000], 3: [0.066,  3.58] } # OPLS C-C

# Bond Coeffs (Harmonic: K [kcal/mol/A^2], r0 [A]) - SPC/Fw values
# K_OH = 1106 kcal/mol/A^2 (converted from 462750 kJ/mol/nm^2)
# r0 = 1.0 A (SPC/E geometry)
bond_coeffs = {
    water_bond_type_id: [1106.0, 1.0000] # <-- K_OH оновлено до SPC/Fw
}

# Angle Coeffs (Harmonic: K [kcal/mol/rad^2], theta0 [degrees]) - SPC/Fw values
# K_HOH = 91.5 kcal/mol/rad^2 (converted from 383 kJ/mol/rad^2)
# theta0 = 109.47 degrees (SPC/E geometry)
angle_coeffs = {
    water_angle_type_id: [91.5, 109.47] # <-- K_HOH оновлено до SPC/Fw
}

# --- Water Topology Finding Parameters ---
min_oh_dist = 0.85; max_oh_dist = 1.15

# --- Helper Function to Read XYZ ---
def read_xyz(filename):
    if not os.path.exists(filename): print(f"ERROR: File not found at {filename}"); sys.exit(1)
    coords_list = []; symbols_list = []
    try:
        with open(filename, "r") as f:
            n_atoms = int(f.readline().strip()); f.readline()
            for i in range(n_atoms):
                line = f.readline(); parts = line.split()
                if len(parts) < 4: raise ValueError(f"Malformed line {i+3}: {line.strip()}")
                symbols_list.append(parts[0].strip().upper()); coords_list.append(list(map(float, parts[1:4])))
        return np.array(coords_list, dtype=float), symbols_list
    except Exception as e: print(f"ERROR reading or parsing XYZ file '{filename}': {e}"); sys.exit(1)

# --- 1. Read Initial Coordinates ---
water_coords_np_orig, water_symbols = read_xyz(water_xyz_file)
graphene_coords_np, graphene_symbols = read_xyz(graphene_xyz_file)
n_water_atoms_read = len(water_symbols)

# --- 2. Center Water Droplet ---
max_graphene_z = 0.0
if len(graphene_coords_np) > 0:
    graphene_center_xy = np.mean(graphene_coords_np[:, :2], axis=0)
    max_graphene_z = np.max(graphene_coords_np[:, 2])
else:
    graphene_center_xy = np.array([0.0, 0.0])

water_coords_np_displaced = np.empty((0,3), dtype=float)
if len(water_coords_np_orig) > 0:
    water_masses = [symbol_to_mass.get(s) for s in water_symbols]
    if None in water_masses: print("ERROR: Mass not found for water atom"); sys.exit(1)
    water_masses_np = np.array(water_masses)
    water_com = np.average(water_coords_np_orig, axis=0, weights=water_masses_np)
    min_water_z_orig = np.min(water_coords_np_orig[:, 2])
    xy_displacement = graphene_center_xy - water_com[:2]
    target_min_water_z = max_graphene_z + desired_z_gap
    z_displacement = target_min_water_z - min_water_z_orig
    water_coords_np_displaced = water_coords_np_orig.copy()
    water_coords_np_displaced[:, :2] += xy_displacement
    water_coords_np_displaced[:, 2] += z_displacement

# --- 3. Process Water Atoms ---
atoms_water = []; oxygens = []; hydrogens = []; atom_id_counter = 1
for i in range(0, n_water_atoms_read - (n_water_atoms_read % 3), 3):
    mol_atoms_temp = []
    try:
        for j in range(3):
            idx = i + j; symbol = water_symbols[idx]; expected_symbol = 'O' if j == 0 else 'H'
            if symbol != expected_symbol: raise ValueError(f"Idx {idx}: Expected '{expected_symbol}' got '{symbol}'")
            type_id = symbol_to_type_id.get(symbol); charge = atom_charge_map.get(symbol)
            if type_id is None or charge is None: raise ValueError(f"Symbol '{symbol}' not in maps.")
            atom = {"id": atom_id_counter + j, "type": type_id, "charge": charge,
                    "x": water_coords_np_displaced[idx, 0], "y": water_coords_np_displaced[idx, 1], "z": water_coords_np_displaced[idx, 2],
                    "mol_id": None}
            mol_atoms_temp.append(atom)
        atoms_water.extend(mol_atoms_temp); oxygens.append(mol_atoms_temp[0]); hydrogens.extend(mol_atoms_temp[1:])
        atom_id_counter += 3
    except Exception as e: pass
last_water_atom_id = atom_id_counter - 1

# --- 4. Determine Water Topology ---
mol_id_counter = 1; bond_id_counter = 1; angle_id_counter = 1; bonds_water = []; angles_water = []
free_hydrogen_ids = set(h["id"] for h in hydrogens); hydrogen_dict = {h["id"]: h for h in hydrogens}; assigned_oxygen_ids = set()
min_r_sq = min_oh_dist**2; max_r_sq = max_oh_dist**2
for oxygen_atom in oxygens:
    if oxygen_atom["id"] in assigned_oxygen_ids: continue
    ox, oy, oz = oxygen_atom["x"], oxygen_atom["y"], oxygen_atom["z"]; nearby_hydrogens = []
    candidate_h_ids = list(free_hydrogen_ids)
    for h_id in candidate_h_ids:
        if h_id not in free_hydrogen_ids: continue
        h_atom = hydrogen_dict[h_id]; dx=h_atom["x"]-ox; dy=h_atom["y"]-oy; dz=h_atom["z"]-oz; r_sq = dx*dx + dy*dy + dz*dz
        if min_r_sq <= r_sq <= max_r_sq: nearby_hydrogens.append((r_sq, h_atom))
    nearby_hydrogens.sort(key=lambda item: item[0])
    if len(nearby_hydrogens) >= 2:
        h1_atom = nearby_hydrogens[0][1]; h2_atom = nearby_hydrogens[1][1]
        current_mol_id = mol_id_counter
        oxygen_atom["mol_id"] = current_mol_id; h1_atom["mol_id"] = current_mol_id; h2_atom["mol_id"] = current_mol_id
        assigned_oxygen_ids.add(oxygen_atom["id"]); free_hydrogen_ids.discard(h1_atom["id"]); free_hydrogen_ids.discard(h2_atom["id"])
        bonds_water.append((bond_id_counter, water_bond_type_id, oxygen_atom["id"], h1_atom["id"])); bond_id_counter += 1
        bonds_water.append((bond_id_counter, water_bond_type_id, oxygen_atom["id"], h2_atom["id"])); bond_id_counter += 1
        angles_water.append((angle_id_counter, water_angle_type_id, h1_atom["id"], oxygen_atom["id"], h2_atom["id"])); angle_id_counter += 1
        mol_id_counter += 1

# --- 5. Process Graphene Atoms ---
atoms_graphene = []; graphene_start_id = last_water_atom_id + 1
type_id_c = symbol_to_type_id.get("C"); charge_c = atom_charge_map.get("C"); mol_id_c = 0; graphene_atom_count = 0
for i, symbol in enumerate(graphene_symbols):
    if symbol != "C": continue
    atom = {"id": graphene_start_id + graphene_atom_count, "type": type_id_c, "charge": charge_c, "mol_id": mol_id_c,
            "x": graphene_coords_np[i, 0], "y": graphene_coords_np[i, 1], "z": graphene_coords_np[i, 2]}
    atoms_graphene.append(atom); graphene_atom_count += 1

# --- 6. Combine Atom Lists and Determine Final Box Size ---
all_atoms = atoms_water + atoms_graphene
n_total_atoms = len(all_atoms); n_total_bonds = len(bonds_water); n_total_angles = len(angles_water)
if not use_hardcoded_box:
    if n_total_atoms == 0: print("ERROR: No atoms found!"); sys.exit(1)
    all_coords_final_np = np.array([[a['x'], a['y'], a['z']] for a in all_atoms], dtype=float)
    min_coords = np.min(all_coords_final_np, axis=0); max_coords = np.max(all_coords_final_np, axis=0)
    xlo, ylo, zlo = min_coords - padding
    xhi, yhi = max_coords[:2] + padding
    zhi = max_coords[2] + z_top_padding
else:
    xlo, xhi = xlo_hardcoded, xhi_hardcoded; ylo, yhi = ylo_hardcoded, yhi_hardcoded; zlo, zhi = zlo_hardcoded, zhi_hardcoded

# --- 7. Write COMPLETE LAMMPS Data File ---
try:
    with open(lammps_data_file, "w") as f:
        f.write("LAMMPS data file generated by python script (FLEXIBLE water, centered, OPLS C)\n\n") # Updated comment
        f.write(f"{n_total_atoms} atoms\n"); f.write(f"{n_total_bonds} bonds\n"); f.write(f"{n_total_angles} angles\n")
        f.write(f"{n_total_dihedrals} dihedrals\n"); f.write(f"{n_total_impropers} impropers\n\n")
        f.write(f"{n_atom_types} atom types\n"); f.write(f"{n_bond_types} bond types\n"); f.write(f"{n_angle_types} angle types\n\n")
        f.write(f"{xlo:.6f} {xhi:.6f} xlo xhi\n"); f.write(f"{ylo:.6f} {yhi:.6f} ylo yhi\n"); f.write(f"{zlo:.6f} {zhi:.6f} zlo zhi\n\n")
        f.write("Masses\n\n");
        for type_id in range(1, n_atom_types + 1):
            _, mass = atom_types_config[type_id-1]; f.write(f"{type_id:<4d} {mass:<10.4f}\n")
        f.write("\n")
        f.write("Pair Coeffs\n\n");
        for type_id in range(1, n_atom_types + 1):
            coeffs = pair_coeffs.get(type_id)
            if coeffs: f.write(f"{type_id:<4d} {coeffs[0]:<12.6f} {coeffs[1]:<10.4f}\n")
            else: f.write(f"{type_id:<4d} {0.0:<12.6f} {0.0:<10.4f}\n")
        f.write("\n")
        # --- Bond Coeffs (Write K and r0 for harmonic) ---
        f.write("Bond Coeffs\n\n");
        for type_id in range(1, n_bond_types + 1):
            coeffs = bond_coeffs.get(type_id) # <-- Get K and r0
            if coeffs: f.write(f"{type_id:<4d} {coeffs[0]:<12.4f} {coeffs[1]:<10.4f}\n") # <-- WRITE BOTH K and r0
            else: print(f"Warning: Missing bond_coeffs for bond type {type_id}")
        f.write("\n")
        # --- Angle Coeffs (Write K and theta0 for harmonic) ---
        f.write("Angle Coeffs\n\n");
        for type_id in range(1, n_angle_types + 1):
            coeffs = angle_coeffs.get(type_id) # <-- Get K and theta0
            if coeffs: f.write(f"{type_id:<4d} {coeffs[0]:<12.4f} {coeffs[1]:<10.2f}\n") # <-- WRITE BOTH K and theta0
            else: print(f"Warning: Missing angle_coeffs for angle type {type_id}")
        f.write("\n")
        # --- Atoms ---
        f.write("Atoms # full\n\n");
        all_atoms.sort(key=lambda a: a['id'])
        for a in all_atoms:
            mol_id_to_write = a['mol_id'] if a['mol_id'] is not None else 0
            f.write(f"{a['id']:<8d} {mol_id_to_write:<8d} {a['type']:<5d} {a['charge']:.4f} {a['x']:16.6f} {a['y']:16.6f} {a['z']:16.6f}\n")
        f.write("\n")
        # --- Bonds ---
        if n_total_bonds > 0:
            f.write("Bonds\n\n"); bonds_water.sort(key=lambda b: b[0])
            for b in bonds_water: f.write(f"{b[0]:<8d} {b[1]:<5d} {b[2]:<8d} {b[3]:<8d}\n")
            f.write("\n")
        # --- Angles ---
        if n_total_angles > 0:
            f.write("Angles\n\n"); angles_water.sort(key=lambda ang: ang[0])
            for ang in angles_water: f.write(f"{ang[0]:<8d} {ang[1]:<5d} {ang[2]:<8d} {ang[3]:<8d} {ang[4]:<8d}\n")
            f.write("\n")
except Exception as e:
    print(f"ERROR writing {lammps_data_file}: {e}"); sys.exit(1)

# Minimal final print statements
print(f"Generated: {lammps_data_file}")

Generated: /mnt/c/Users/vbarv/Desktop/course/project/lammps/data/system_pillars.data
